#### Importing Libraries

In [90]:
# Setup & Imports
import re
import random
import pandas as pd
import json

##### Creating empty dictionary

In [91]:
# Step-by-step creation of the ICU Data Dictionary

# 1️⃣ Start with an empty dictionary
icu_data_dict_granular = {}

# 1. Patient Identifiers

## 1.1. Name and Surname (Full Match)

### 1.1.1. Build Pattern with live iteraction


Using regex101 to iterate over pattern possibilities: 
https://regex101.com/


### 1.1.2. Commented Identifier Specificities

In [92]:
# Name could include the word "paciente" (patient), which in portuguese has no gender inflection. It could read just "nome" (name) or "nome completo" (complete name).
Field_Identifier = [r"\b(?:[Nn]ome)\b",
                    r"[nN]ome\s?[cC]ompleto",
                    r"\b[Nn]ome\s?(?:d[oa])?\s?[Pp]aciente\b",
                    r"[Pp]aciente"]

# 1. This is a Regex to perform a FULL MATCH, checking names that are minimally complete: at least one first name and one last name, with the widest possible range of possible patterns.
# 2. Names in Brazil can be quite creative (unlike Portugal, which has a naming law with a list of possible names), so we need to account for many variations in Brazil.
# 3. There is a peculiarity in Portuguese names, which is the presence of possessive particles ("de", "do", "da") to indicate belonging to a particular family. I have also included the possessive particles from other languages, such as Arabic, Dutch, Italian, German and Spanish, since we have had considerable immigration from these groups to Brazil in recent centuries".
# 4. I added specific accents that correspond to Portuguese, such as circunflex "^" and tilda "~", and also took into account possible typos in the accentuation, which are common.
# 5. Added a possible hyphen to separate compound names in some cases.
# 6. Given that we're fully congruent, it's really important not to ignore capitalisation.
Content_Identifier =  (
    r"\b[A-ZÀ-Ÿ][a-zà-ÿ]+\s?-?"
    r"(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|"
    r"\bvon der\b|\bvan der\b|\bvan den\b|du|\bde la\b|al|bin|ibn|el)?"
    r"\s?[A-ZÀ-Ÿ][a-zà-ÿ]+\s?-?"
    r"(?:(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|"
    r"\bvon der\b|\bvan der\b|\bvan den\b|du|\bde la\b|al|bin|ibn|el)?"
    r"\s?-?[A-ZÀ-Ÿ][a-zà-ÿ]+\s?)*\b"
)


#### 1.1.3. Positive Stress-Test (PST)

Positive Stress-Test aims at seeing how well the identifier fits to different cases (generality).

In [93]:
# Define components of Portuguese full names
first_names = ["João", "Maria", "Carlos", "Fernanda", "Antônio", "Ênio", "Beatriz", "Luís", "Ana", "Paulo", "Júlia", "Eduardo"]
middle_names = ["José", "Clara", "Júnior", "Paulo", "Pedro", "Silva", "Jara", "Miguel", "Henrique", "Neves", "Vales"]
last_names = ["Santos", "Costa", "Souza", "Nascimento", "Pereira", "Oliveira", "Vales", "Ribeiro", "Mendes", "Rodrigues"]
particles = ["da", "de", "do", "dos", "das", "di", "von der","al", "ibn", "von"]

# Generate random full names with variation
def generate_portuguese_names(n):
    names = []
    for _ in range(n):
        first = random.choice(first_names)
        middle = random.choice(middle_names) if random.random() > 0.5 else ""
        particle = random.choice(particles) if random.random() > 0.3 else ""
        last = random.choice(last_names)
        hyphen = "-" if random.random() > 0.8 else " "
        
        # Construct name with variations
        full_name = first
        if middle:
            full_name += hyphen + middle if random.random() > 0.7 else " " + middle
        if particle:
            full_name += " " + particle
        full_name += " " + last

        # Add diacritic variations randomly
        full_name = full_name.replace("a", "à") if random.random() > 0.9 else full_name
        full_name = full_name.replace("o", "ô") if random.random() > 0.9 else full_name
        full_name = full_name.replace("e", "ê") if random.random() > 0.9 else full_name

        names.append(full_name)
    return names

# Generate test dataset
test_names = generate_portuguese_names(100)

# Define the regex pattern for full names
name_regex_test = re.compile(
    r"\b[A-ZÀ-Ÿ][a-zà-ÿ]+\s?-?(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|de la|al|bin|ibn|el)?\s?[A-ZÀ-Ÿ][a-zà-ÿ]+\s?-?(?:(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\bvon der\b|\bvan der\b|\bvan den\b|du|\bde la\b|al|bin|ibn|el)?\s?-?[A-ZÀ-Ÿ][a-zà-ÿ]+\s?)*\b",

)

# Test regex against generated names
matches_test = [name for name in test_names if name_regex_test.fullmatch(name)]

# Display results
df_results_test = pd.DataFrame({"Generated Name": test_names, "Matched": ["✅" if name in matches_test else "❌" for name in test_names]})


In [94]:
df_results_test.style #The names that failed are not "real names", so the identifier seems to perform well under PST.

,Generated Name,Matched
0,Maria Neves von der Mendes,✅
1,Bêatriz al Pêrêira,✅
2,Ana al Santos,✅
3,Antônio Clara do Mendes,✅
4,Ana José Pereira,✅
5,Beatriz dos Rodrigues,✅
6,Carlos Silva das Mendes,✅
7,Beatriz da Costa,✅
8,Luís Jara ibn Ribeiro,✅
9,Eduardo Paulo dos Souza,✅


### 1.1.3. Negative Stress-Test (NST)

Checks that the identifier is not too broad and matches what it shouldn't (specificity).

In [95]:
# Define Negative Stress-Test Cases (NST)
nst_cases = [
    # Institutional & Business Names
    "Hospital Santa Maria",
    "Clínica Oliveira & Associados",
    "Laboratório São Paulo",
    "Empresa Pereira Consultoria",
    "Restaurante Mendes Gourmet",
    
    # Partial & Altered Names
    "Paciente: João, Silva",
    "Pereira da Silva",
    "João 12345",
    "Von Ribeiro",
    "Antônio e sua equipe médica",
    
    # Address-Like Structures
    "Rua João Batista, 250",
    "Avenida da Costa",
    "Estrada do Ribeiro",
    "Travessa Doutor Mendes",
    "Rodovia Fernandes & Associados",
    
    # Numbers That Look Like Identifiers
    "Pedido Nº 123456",
    "Código de Produto: 9876543210",
    "Referência: 000000112233",
    "ID Transação: 573829",
    "Telefone: (11) 91234-5678",
    
    # General Medical Terms (No Patient Data)
    "Estudos indicam aumento de infartos no Brasil",
    "A hipertensão é uma condição comum em idosos",
    "Novo medicamento para diabetes foi aprovado",
    "Uso de antibióticos cresce no país",
    "Conferência médica aborda diagnóstico precoce",
    
    # Medical Procedures Without Patient-Specific Data
    "Manual técnico sobre ventilação mecânica",
    "Guia de conduta médica para tratamento de sepsis",
    "Regulamentação sobre drogas de infusão contínua",
    "Revisão científica sobre conduta médica na UTI",
    "Exame físico na triagem hospitalar",
    
    # Discussion of Medical Topics Without Patient Data
    "Emergência de trânsito encaminha vítimas para hospital",
    "Notícias sobre novos tratamentos para câncer",
    "Governo lança campanha de prevenção a doenças cardíacas",
    "Congresso médico discute novas abordagens terapêuticas",
    "Professor fala sobre evolução dos cuidados intensivos",
    
    # Miscellaneous (Non-Patient Data)
    "Pesquisadores buscam vacinas para novas epidemias",
    "Sistema hospitalar recebe atualização tecnológica",
    "Assistência social ajuda famílias de internados",
    "Guia de administração hospitalar publicado este mês",
    "Novos protocolos de atendimento emergencial foram adotados",
    
    # Financial & Insurance Terms Without PII
    "Convênio cobre internação de pacientes",
    "Planos de saúde ajustam valores para 2024",
    "Tabela de preços para exames laboratoriais atualizada",
    "Reembolso de consultas médicas pelo convênio",
    "Novas regras para atendimento de urgência no SUS",
]

# Apply regex and store results
nst_results = ["✅ Matched" if re.fullmatch(Content_Identifier, case) else "❌ Ignored" for case in nst_cases]

# Create and display results DataFrame
df_nst_results = pd.DataFrame({"NST Test Case": nst_cases, "Detection Result": nst_results})
df_nst_results.style

,NST Test Case,Detection Result
0,Hospital Santa Maria,✅ Matched
1,Clínica Oliveira & Associados,❌ Ignored
2,Laboratório São Paulo,✅ Matched
3,Empresa Pereira Consultoria,✅ Matched
4,Restaurante Mendes Gourmet,✅ Matched
5,"Paciente: João, Silva",❌ Ignored
6,Pereira da Silva,✅ Matched
7,João 12345,❌ Ignored
8,Von Ribeiro,✅ Matched
9,Antônio e sua equipe médica,❌ Ignored


### 1.1.4. Moriarty Stress-Test (MST)

In [96]:
compiled_pattern = re.compile(Content_Identifier, re.IGNORECASE)

# Define MST cases - Adversarial Evasion Attempts
mst_cases = [
    # 1️⃣ **Spacing & Formatting Manipulation**
    "J o ã o   S i l v a",
    "M a r i a - C l a r a",

    # 2️⃣ **Obfuscation via Special Characters**
    "Jo@o S!lva",
    "Mária_Clára",

    # 3️⃣ **Encoding Tricks**
    "J%6F%61o%20Silva",
    "Maria\x20\x2D\x20Clara",
    "Carlos\u00A0Mendes",

    # 4️⃣ **Splitting Information Over Multiple Lines**
    "João\nSilva",
    "Fernanda\nCosta",

    # 5️⃣ **Homoglyph Substitutions (Look-Alike Characters)**
    "Jоãо Silva",  # Cyrillic 'о' (not Latin 'o')
    "Mαria Clαra",  # Greek 'α' instead of 'a'
    "Luís Mendes",

    # 6️⃣ **Concatenation with Other Entities**
    "Nome: João Silva, Cargo: Médico",
    "Paciente: Maria Clara - Leito 25",
    "Identificação: Carlos Mendes - Unidade 3",

    # 7️⃣ **Non-traditional Separators**
    "João•Silva",
    "Maria‧Clara",  # Using a middle dot
    "Carlos_Mendes",

    # 8️⃣ **Partial Matches with Extra Context**
    "O paciente João Silva está em observação.",
    "Maria Clara recebeu alta ontem.",
]

# Apply MST regex testing
mst_results = [
    "✅ Matched" if compiled_pattern.search(case) else "❌ Evaded"
    for case in mst_cases
]

# Create MST results DataFrame
df_mst_results = pd.DataFrame({"MST Test Case": mst_cases, "Detection Result": mst_results})
df_mst_results.style

,MST Test Case,Detection Result
0,J o ã o S i l v a,❌ Evaded
1,M a r i a - C l a r a,❌ Evaded
2,Jo@o S!lva,❌ Evaded
3,Mária_Clára,❌ Evaded
4,J%6F%61o%20Silva,❌ Evaded
5,Maria - Clara,✅ Matched
6,Carlos Mendes,✅ Matched
7,João Silva,✅ Matched
8,Fernanda Costa,✅ Matched
9,Jоãо Silva,✅ Matched


### 1.1.5. Time Stress-Test (MST)

Names take hundreds of years to change patterns. If still seeking full match, we can review it each year to account intense immigration in the recent decades.

### 1.1.6. Add identifier to dict

In [97]:
# Initialize ICU Dictionary if not already present
icu_data_dict_granular = icu_data_dict_granular if 'icu_data_dict_granular' in locals() else {}

# Define categories and their respective identifiers
icu_categories = [
    ("Patient_Identifiers", "Complete_Name", Field_Identifier, Content_Identifier, "Detects full patient names using regex-based entity recognition.")
 ]

# Add each field identifier, content regex, and DLP strategy to the dictionary
for category, field, field_identifier, content_identifier, dlp_strategy in icu_categories:
    icu_data_dict_granular.setdefault(category, {}).setdefault("Fields", {})[field] = {
        "Field_Identifier": field_identifier,
        "Content_Identifier": content_identifier,
        "DLP_Strategy": dlp_strategy
    }

# Print the structured ICU dictionary
import json
print(json.dumps(icu_data_dict_granular, indent=4, ensure_ascii=False))


{
    "Patient_Identifiers": {
        "Fields": {
            "Complete_Name": {
                "Field_Identifier": [
                    "\\b(?:[Nn]ome)\\b",
                    "[nN]ome\\s?[cC]ompleto",
                    "\\b[Nn]ome\\s?(?:d[oa])?\\s?[Pp]aciente\\b",
                    "[Pp]aciente"
                ],
                "Content_Identifier": "\\b[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?-?(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\\bvon der\\b|\\bvan der\\b|\\bvan den\\b|du|\\bde la\\b|al|bin|ibn|el)?\\s?[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?-?(?:(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\\bvon der\\b|\\bvan der\\b|\\bvan den\\b|du|\\bde la\\b|al|bin|ibn|el)?\\s?-?[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?)*\\b",
                "DLP_Strategy": "Detects full patient names using regex-based entity recognition."
            }
        }
    }
}


## 1.2. Hospital Admitance/Doctor Record Id

### 1.2.1. Build Pattern

In [98]:
#There are many ways to abbreviate number (N./n./nº/#)
# Alternative words for número could be used such as "ID", "identificação" (identification), "código" (code).
# Inserted a kind of idiomatic variation that inverts the word order, instead of "número de atendimento" (attendance number) saying "atendimento número" (number attendance).
# We could also have an Id for "prontuário" (medical record number)
Field_Identifier = [
    r"\b(?:n[úu]mero|N\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\s?(?:de)?\s?atendimento\b",
    r"\batendimento\s?(?:n[úu]mero|N\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\b",
    r"\b(?:n[úu]mero|N\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\s?(?:do)?\s?Prontuário\b",
    r"\b(?:Prontuário\s?n[úu]mero|N\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\b",  
       ]

# Patient ID is usually a 5 to 10-digit number, could start with 2 letters... This would involve understanding the hospital's ID generation pattern in the "preventive alignment" phase of the methodology.
Content_Identifier = r"\b\[A-Z]{2}\d{5,10}\b"


### 1.2.2. Positive Stress-Test (PST)

In [99]:
# Will focus on stress-testing the identifier field this time
pst_cases = [
    "Número de Atendimento: AB123456",
    "Paciente com ID 987654 está na UTI.",
    "CÓDIGO de atendimento 56789 emitido.",
    "Identificação do Prontuário: 874512",
    "O paciente tem o # atendimento 23456FG.",
    "O Número do prontuário foi registrado corretamente.",
    "ID do paciente: 1029384756",
    "CÓDIGO de IDENTIFICAÇÃO: 6758493021",
    "Atendimento nº 765432 no hospital São Paulo.",
    "Identificação do paciente: 495039204",
]

# Apply regex and store results
pst_results = [
    "✅ Matched" if any(re.search(pattern, case, re.IGNORECASE) for pattern in Field_Identifier) else "❌ Missed"
    for case in pst_cases
]

# Create and display results DataFrame
df_pst_results = pd.DataFrame({"PST Test Case": pst_cases, "Detection Result": pst_results})
df_pst_results

,PST Test Case,Detection Result
0,Número de Atendimento: AB123456,✅ Matched
1,Paciente com ID 987654 está na UTI.,✅ Matched
2,CÓDIGO de atendimento 56789 emitido.,✅ Matched
3,Identificação do Prontuário: 874512,✅ Matched
4,O paciente tem o # atendimento 23456FG.,❌ Missed
5,O Número do prontuário foi registrado corretam...,✅ Matched
6,ID do paciente: 1029384756,✅ Matched
7,CÓDIGO de IDENTIFICAÇÃO: 6758493021,✅ Matched
8,Atendimento nº 765432 no hospital São Paulo.,✅ Matched
9,Identificação do paciente: 495039204,✅ Matched


### 1.2.3. Negative Stress-Test (NST)

In [100]:
# --- 2️⃣ Negative Stress-Test (NST) ---
nst_cases = [
    "O hospital recebeu uma nova remessa de seringas.",
    "Equipe de enfermagem realizou treinamento em cuidados intensivos.",
    "Lista de equipamentos disponíveis no setor de emergência.",
    "Reunião com fornecedores para aquisição de novos insumos.",
    "Procedimentos administrativos para solicitação de medicamentos.",
    "Paciente não apresentou necessidade de medicação adicional.",
    "A farmácia hospitalar organiza os estoques semanalmente.",
    "Discussão sobre melhores práticas em prescrição médica.",
    "Professor de farmacologia apresenta nova pesquisa.",
    "Relatório sobre uso racional de medicamentos na UTI.",
    "Insumos médicos estão disponíveis para enfermagem.",
    "Política de segurança hospitalar reforçada.",
    "Gestão hospitalar planeja novas estratégias para atendimento.",
    "Pesquisa avalia eficácia de tratamento não farmacológico.",
    "Artigo publicado sobre tendências em terapias médicas.",
    "Especialistas discutem avanços na área de anestesiologia.",
    "A farmácia clínica é essencial para otimizar prescrições.",
    "Engenharia hospitalar realiza manutenção em ventiladores.",
    "Comissão de controle de infecção analisa novos protocolos.",
    "Hospital inaugura novo laboratório de análises clínicas.",
]

# Apply regex correctly for a list of patterns
nst_results = [
    "✅ Matched" if any(re.search(pattern, case, re.IGNORECASE) for pattern in Field_Identifier) 
    else "❌ Correctly Ignored"
    for case in nst_cases
]

# Create and display results DataFrame
df_nst_results = pd.DataFrame({"NST Test Case": nst_cases, "Detection Result": nst_results})
df_nst_results.style


,NST Test Case,Detection Result
0,O hospital recebeu uma nova remessa de seringas.,❌ Correctly Ignored
1,Equipe de enfermagem realizou treinamento em cuidados intensivos.,❌ Correctly Ignored
2,Lista de equipamentos disponíveis no setor de emergência.,❌ Correctly Ignored
3,Reunião com fornecedores para aquisição de novos insumos.,❌ Correctly Ignored
4,Procedimentos administrativos para solicitação de medicamentos.,❌ Correctly Ignored
5,Paciente não apresentou necessidade de medicação adicional.,❌ Correctly Ignored
6,A farmácia hospitalar organiza os estoques semanalmente.,❌ Correctly Ignored
7,Discussão sobre melhores práticas em prescrição médica.,❌ Correctly Ignored
8,Professor de farmacologia apresenta nova pesquisa.,❌ Correctly Ignored
9,Relatório sobre uso racional de medicamentos na UTI.,❌ Correctly Ignored


### 1.2.4. Moriarty Stress-Test (MST)

In [101]:
# Define MST cases - Adversarial Evasion Attempts
mst_cases = [
    # 1️⃣ **Spacing & Formatting Manipulation**
    "N ú m e r o    d e    A t e n d i m e n t o :  1 2 3 4 5 6",
    "Prontuário   :   987 654",
    "Identificação - Paciente 102938",

    # 2️⃣ **Obfuscation via Special Characters**
    "Nº Atendimento: @123456!",
    "ID: 9*8*7*6*5*4",
    "Código do paciente: 5_6_7_8_9",

    # 3️⃣ **Encoding Manipulations**
    "N%FAmero%20de%20Atendimento%20:%20123456",
    "Prontuario\x20\x3A\x20987654",
    "Identifica\u00E7\u00E3o%20do%20paciente%20%3A%20102938",

    # 4️⃣ **Splitting Information Over Multiple Lines**
    "Número de\nAtendimento: 123456",
    "Paciente\nID:\n987654",
    "Código\ndo\nPaciente:\n102938",

    # 5️⃣ **Homoglyph Substitutions (Look-Alike Characters)**
    "Nuмero de Atendιmento: 123456",  # Cyrillic 'м' and Greek 'ι'
    "Ρrontuário: 987654",  # Greek 'Ρ' (Rho) instead of 'P'
    "Identificațião do paciente: 102938",  # Romanian 'ț' instead of 't'
]

# Apply MST regex testing
mst_results = [
    "✅ Matched" if any(re.search(pattern, case, re.IGNORECASE) for pattern in Field_Identifier) else "❌ Evaded" for case in mst_cases
]

# Create MST results DataFrame
df_mst_results = pd.DataFrame({"MST Test Case": mst_cases, "Detection Result": mst_results})
df_mst_results.style


,MST Test Case,Detection Result
0,N ú m e r o d e A t e n d i m e n t o : 1 2 3 4 5 6,✅ Matched
1,Prontuário : 987 654,❌ Evaded
2,Identificação - Paciente 102938,✅ Matched
3,Nº Atendimento: @123456!,✅ Matched
4,ID: 9*8*7*6*5*4,✅ Matched
5,Código do paciente: 5_6_7_8_9,✅ Matched
6,N%FAmero%20de%20Atendimento%20:%20123456,✅ Matched
7,Prontuario : 987654,❌ Evaded
8,Identificação%20do%20paciente%20%3A%20102938,✅ Matched
9,Número de Atendimento: 123456,✅ Matched


### 1.2.5. Time Stress-Test (TST) Considerations



The identification of **patient numbers and medical identifiers** requires **long-term adaptability**, given that:
1. **Hospital record-keeping evolves** → Different hospitals may **adopt new numbering conventions**.
2. **Policy and compliance changes** → National or institutional **data security policies** might impose **new formats**.
3. **EHR (Electronic Health Records) integration** → New **interoperability standards** could **alter field names** or introduce **structured formats**.

To maintain robustness:
- **Automated Periodic Audits** → Ensure regex remains effective by **validating against evolving real-world ICU data**.
- **Historical Change Tracking** → Log detected identifiers over time to **spot patterns & shifts**.
- **Proactive Adjustment Mechanisms** → Flag identifiers **at risk of obsolescence** and schedule **review cycles** accordingly.

"""

### 1.2.6. Add Identifiers to Dict

In [102]:
# Initialize ICU Dictionary if not already present
icu_data_dict_granular = icu_data_dict_granular if 'icu_data_dict_granular' in locals() else {}

# Define categories and their respective identifiers
icu_categories = [
    ("Patient_Identifiers", "Patient_Number", Field_Identifier, Content_Identifier, 
     "Detects numerical patient identifiers in hospital records.")
]

# Add each field identifier, content regex, and DLP strategy to the dictionary
for category, field, field_identifier, content_identifier, dlp_strategy in icu_categories:
    icu_data_dict_granular.setdefault(category, {}).setdefault("Fields", {})[field] = {
        "Field_Identifier": field_identifier,
        "Content_Identifier": content_identifier,
        "DLP_Strategy": dlp_strategy
    }

# Print the structured ICU dictionary
import json
print(json.dumps(icu_data_dict_granular, indent=4, ensure_ascii=False))


{
    "Patient_Identifiers": {
        "Fields": {
            "Complete_Name": {
                "Field_Identifier": [
                    "\\b(?:[Nn]ome)\\b",
                    "[nN]ome\\s?[cC]ompleto",
                    "\\b[Nn]ome\\s?(?:d[oa])?\\s?[Pp]aciente\\b",
                    "[Pp]aciente"
                ],
                "Content_Identifier": "\\b[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?-?(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\\bvon der\\b|\\bvan der\\b|\\bvan den\\b|du|\\bde la\\b|al|bin|ibn|el)?\\s?[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?-?(?:(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\\bvon der\\b|\\bvan der\\b|\\bvan den\\b|du|\\bde la\\b|al|bin|ibn|el)?\\s?-?[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?)*\\b",
                "DLP_Strategy": "Detects full patient names using regex-based entity recognition."
            },
            "Patient_Number": {
                "Field_Identifier": [
                    "\\b(?:n[úu]mero|N\\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\\s?(?:de)?\\s?atendimento\\

# 2. Clinical Medical Data

## 2.1. Diagnosis

### 2.1.1. Build Pattern

In [103]:
# 1. I have researched the most common causes of admission to the ICU so that I can still use identifiers to capture such cases without having to use computationally intensive resources.
# 2. Sepsis is a good example to demonstrate the linguistics-first approach, and it is one of the most common reasons for admission to the ICU.
# 3. The word sepsis comes from σῆψις in ancient Greek, which means to putrefy or decay, in Portuguese it could have the following variations:
## 3.1. Sepse (Portuguese variant), sepsis (Latin variant), séptico (that which has its origin in sepsis), septicemia (referring to the whole "sepsis process/diagnosis"). 
## 3.2. Nowadays, sepsis could simply be called "infecção generalizada" (generalized infection).
## 3.3. I will not simply use the word stem "sep-", because it could be overlapped with words related to the number seven, since we have Latin variations of "septem" or "septimus", such as "septuagenário", a person who is more than 70 years old.



Guide to nurses for most frequent cases in ICUs: https://www.freshrn.com/most-common-icu-admission-diagnosis/ <br>
Pubmed publication for most common ICU admission cases in Brazil: https://pmc.ncbi.nlm.nih.gov/articles/PMC8889595/

In [104]:
#Researched all the variations possible for "Diagnóstico" (diagnosis)
Field_Identifier = r"\b(?:Diagn[óo]stico|\bCID-?10\b|Doen[çc]a|Enfermidade|Patologia|Condi[çc][ãa]o|S[ií]ndrome|Achado Cl[ií]nico|Comorbidade|Progn[óo]stico|Classifica[çc][ãa]o|Quadro\s?Clínico)\b",
Content_Identifier = (
    r"(?:"
    r"S[eé]p(?:tico|ticemia|sis|se)s?|"
    r"neuro|cereb(?:ral|ro)|card|"
    r"infec[cç][aã]o|"
    r"Pneumonia|"
    r"Hemorragia|"
    r"Síndrome|"
    r"Hipertens[ãa][oae]|"
    r"Diabetes|"
    r"AVC|aneurisma|Enfart[eo]|"
    r"\bInsufici[êe]ncia\s?Renal\b|"
    r"\bFal[êe]ncia\s?Hep[áa]tica\b|"
    r"\bLes[ãa]o\s?Renal\s?Aguda\b|"
    r"\bFal[êe]ncia\s?(?:de M[úu]ltiplos|múltipla de)\s?[Óo]rg[ãa]os\b|"
    r"\bEmbolia\s?Pulmonar\b|"
    r"\bInsufici[êe]ncia\s?Respirat[óo]ria\b|"
    r"\bFal[êe]ncia\s?Card[íi]aca\b|"
    r"\bParada\s?Card[íi]aca\b"
    r")"
)


### 2.1.2. Positive Stress-Test (PST)

In [105]:
pst_cases = [
    "Paciente com sepse grave", "Diagnóstico: Pneumonia adquirida na comunidade",
    "Infecção generalizada", "CID-10: I10 - Hipertensão essencial",
    "Aneurisma cerebral", "Insuficiência renal aguda",
    "Parada cardiorrespiratória", "Falência múltipla de órgãos",
    "Embolia pulmonar maciça", "Doença cardíaca isquêmica",
    "Insuficiência respiratória grave", "Comorbidade de diabetes e hipertensão",
    "Paciente com AVC hemorrágico", "Síndrome metabólica avançada",
    "Diagnóstico confirmado de encefalopatia hepática",
    "Lesão renal aguda devido a sepse", "Septicemia neonatal",
    "Hipertensão pulmonar secundária", "Falência hepática fulminante",
    "Infecção urinária complicada", "Paciente com quadro de choque séptico",
    "Diagnóstico de cardiomiopatia dilatada", "Edema agudo de pulmão",
    "Hemorragia intracraniana", "Paciente apresenta falência respiratória",
    "História de insuficiência cardíaca descompensada",
    "Paciente diabético descompensado", "Doença pulmonar obstrutiva crônica",
    "Infecção pós-operatória grave", "Paciente internado por choque cardiogênico",
    "Hipotensão refratária", "Paciente apresenta pneumonia nosocomial",
    "Embolia cerebral", "Síndrome do desconforto respiratório agudo",
    "Complicação de insuficiência hepática", "Paciente com diagnóstico de pericardite",
    "Parada cardíaca revertida", "Infarto agudo do miocárdio",
    "Diagnóstico diferencial entre encefalopatia e meningite",
    "Paciente com hipertensão maligna", "Paciente internado com insuficiência renal terminal",
    "Doença pulmonar intersticial avançada", "Paciente em coma hepático",
    "História de arritmia ventricular complexa", "Paciente com crise hipertensiva",
    "Diagnóstico de insuficiência cardíaca congestiva",
    "Infecção de corrente sanguínea associada a cateter", "Paciente com quadro de meningite bacteriana",
    "Paciente com falência de múltiplos órgãos após trauma",
]


# Apply regex
pst_results = ["✅ Matched" if re.search(Content_Identifier, case, re.IGNORECASE) else "❌ Missed" for case in pst_cases]

# Display results
df_pst_results = pd.DataFrame({"PST Test Case": pst_cases, "Detection Result": pst_results})
df_pst_results


,PST Test Case,Detection Result
0,Paciente com sepse grave,✅ Matched
1,Diagnóstico: Pneumonia adquirida na comunidade,✅ Matched
2,Infecção generalizada,✅ Matched
3,CID-10: I10 - Hipertensão essencial,✅ Matched
4,Aneurisma cerebral,✅ Matched
5,Insuficiência renal aguda,✅ Matched
6,Parada cardiorrespiratória,✅ Matched
7,Falência múltipla de órgãos,✅ Matched
8,Embolia pulmonar maciça,✅ Matched
9,Doença cardíaca isquêmica,✅ Matched


### 2.1.3. Negative Stress-Test(NST)

In [106]:
nst_cases = [
    "Paciente apresentou melhora clínica", "Alta hospitalar programada para amanhã",
    "Tomografia do crânio sem alterações significativas",
    "Paciente sem histórico de doenças crônicas", "Necessidade de ventilação mecânica avaliada",
    "Discussão sobre novas diretrizes para sepse", "Estudos indicam aumento da resistência bacteriana",
    "Artigo científico sobre embolia pulmonar", "Hospital adquiriu novos monitores cardíacos",
    "Paciente transferido para unidade de cuidados intermediários",
    "Técnicas avançadas de suporte ventilatório", "Não há sinais de infecção ativa",
    "Discussão de casos clínicos sobre hipertensão", "Histórico de infecções prévias analisado",
    "Paciente encaminhado para fisioterapia respiratória",
    "Enfermagem monitora sinais vitais regularmente", "Paciente será submetido a cirurgia eletiva",
    "Evolução clínica satisfatória após antibiótico", "Paciente sem necessidade de diálise no momento",
    "Nova pesquisa sobre falência cardíaca em idosos", "Medicação ajustada conforme evolução do quadro",
    "Paciente acordado e orientado, sem déficits neurológicos",
    "Avaliação de risco para embolia pulmonar sem achados clínicos",
    "Nenhuma alteração no eletrocardiograma do paciente",
    "Necessidade de internação hospitalar reavaliada",
    "Paciente sem queixas de dor torácica", "Protocolo de prevenção de infecções hospitalares atualizado",
    "Treinamento da equipe médica sobre sepse", "Não há sinais de agravamento da função hepática",
    "Discussão sobre manejo clínico da hipertensão", "Paciente sem necessidade de intervenção cirúrgica",
    "Paciente será avaliado para alta da UTI", "Nenhuma evidência de sepse ou infecção",
    "Resultados laboratoriais sem alterações relevantes",
    "Conferência médica discute tratamentos inovadores",
    "Paciente permanece hemodinamicamente estável",
    "Análise estatística sobre taxas de internação hospitalar",
    "Paciente sem sinais de desconforto respiratório",
    "Análise retrospectiva de casos de insuficiência cardíaca",
    "Paciente apresenta boa resposta ao tratamento",
    "Discussão sobre novas diretrizes para choque séptico",
    "Equipe médica realiza estudo comparativo sobre pneumonia",
    "Paciente sem queixas e com sinais vitais normais",
    "Avaliação médica descarta necessidade de ventilação mecânica",
    "Progresso do paciente avaliado como satisfatório",
    "Pesquisa clínica sobre fatores de risco para diabetes",
    "Resultados de exames descartam patologia grave",
    "Paciente recebe alta após monitoramento na UTI",
]


nst_results = ["❌ Correctly Ignored" if not re.search(Content_Identifier, case, re.IGNORECASE) else "⚠️ FALSE POSITIVE" for case in nst_cases]
df_nst_results = pd.DataFrame({"NST Test Case": nst_cases[:100], "Detection Result": nst_results[:100]})
df_nst_results.style

,NST Test Case,Detection Result
0,Paciente apresentou melhora clínica,❌ Correctly Ignored
1,Alta hospitalar programada para amanhã,❌ Correctly Ignored
2,Tomografia do crânio sem alterações significativas,❌ Correctly Ignored
3,Paciente sem histórico de doenças crônicas,❌ Correctly Ignored
4,Necessidade de ventilação mecânica avaliada,❌ Correctly Ignored
5,Discussão sobre novas diretrizes para sepse,⚠️ FALSE POSITIVE
6,Estudos indicam aumento da resistência bacteriana,❌ Correctly Ignored
7,Artigo científico sobre embolia pulmonar,⚠️ FALSE POSITIVE
8,Hospital adquiriu novos monitores cardíacos,⚠️ FALSE POSITIVE
9,Paciente transferido para unidade de cuidados intermediários,❌ Correctly Ignored


### 2.1.4. Moriarty Stress-Test (MST)

In [107]:
mst_cases = [
    # 1️⃣ Contextual Evasion
    "O exame revelou um quadro típico de Sepse, segundo a análise médica.",
    "Foi prescrito um tratamento emergencial devido ao risco de parada cardíaca.",

    # 2️⃣ Structured Data Evasion
    "Paciente: João \n Diagnóstico:\n\n Sepse severa.",
    "Nome: João | Diagnóstico: AVC | Tratamento: Heparina",
    "Paciente-Estado: ParadaCardiaca",

    # 3️⃣ Semantic Substitution
    "O paciente apresentou um quadro clínico compatível com choque séptico.",
    "Hipotensão grave associada a infecção sistêmica.",

    # 4️⃣ Encoding Tricks
    "Pneumonia".encode('utf-8').hex(),  # Stored as hex encoding
    "P%6E%65umonia",  # URL encoding attempt
    bytes("Pneumonia", "utf-8").decode("unicode_escape"),  # Unicode escape sequences
    "Pneu\x6dm\x6eia",  # Hex-like obfuscation
    "Pneum&#111;nia"  # HTML entity encoding attempt
]

# Apply regex
mst_results = ["✅ Matched" if re.search(Content_Identifier, case, re.IGNORECASE) else "❌ Evaded" for case in mst_cases]

# Display results
df_mst_results = pd.DataFrame({"MST Test Case": mst_cases, "Detection Result": mst_results})
df_mst_results

df_mst_results


,MST Test Case,Detection Result
0,"O exame revelou um quadro típico de Sepse, seg...",✅ Matched
1,Foi prescrito um tratamento emergencial devido...,✅ Matched
2,Paciente: João \n Diagnóstico:\n\n Sepse severa.,✅ Matched
3,Nome: João | Diagnóstico: AVC | Tratamento: He...,✅ Matched
4,Paciente-Estado: ParadaCardiaca,✅ Matched
5,O paciente apresentou um quadro clínico compat...,✅ Matched
6,Hipotensão grave associada a infecção sistêmica.,✅ Matched
7,506e65756d6f6e6961,❌ Evaded
8,P%6E%65umonia,❌ Evaded
9,Pneumonia,✅ Matched


### 2.1.5. Time Stress-Test (TST)

#### TST Goals:
- **Detect linguistic drift** – Identify when new terms appear or old ones become obsolete.
- **Validate long-term robustness** – Ensure the regex captures relevant diagnoses over time.
- **Trigger review alerts** – Raise warnings if medical dictionaries update with new critical conditions.

#### TST Implementation:
- **Automated Monitoring** → Periodically compare regex matches against **ICD-10, SNOMED-CT, and real ICU records** to detect gaps.
- **Historical Tracking** → Log **term frequency** in hospital datasets to monitor shifts in **ICU admission causes**.


### 2.1.6. Add Identifier to Dict

In [108]:
# Initialize ICU Dictionary if not already present
if 'icu_data_dict_granular' not in locals():
    icu_data_dict_granular = {}

# Define categories and their respective identifiers (must be a list of tuples)
icu_categories = [
    ("Medical_Conditions", "Diagnosis", Field_Identifier, Content_Identifier, 
     "Identifies medical conditions, ICD-10 codes, and diagnostic terms in patient records.")
]

# Add each field identifier, content regex, and DLP strategy to the dictionary
for category, field, field_identifier, content_identifier, dlp_strategy in icu_categories:
    icu_data_dict_granular.setdefault(category, {}).setdefault("Fields", {})[field] = {
        "Field_Identifier": field_identifier,
        "Content_Identifier": content_identifier,
        "DLP_Strategy": dlp_strategy
    }

# Print the structured ICU dictionary
import json
print(json.dumps(icu_data_dict_granular, indent=4, ensure_ascii=False))


{
    "Patient_Identifiers": {
        "Fields": {
            "Complete_Name": {
                "Field_Identifier": [
                    "\\b(?:[Nn]ome)\\b",
                    "[nN]ome\\s?[cC]ompleto",
                    "\\b[Nn]ome\\s?(?:d[oa])?\\s?[Pp]aciente\\b",
                    "[Pp]aciente"
                ],
                "Content_Identifier": "\\b[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?-?(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\\bvon der\\b|\\bvan der\\b|\\bvan den\\b|du|\\bde la\\b|al|bin|ibn|el)?\\s?[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?-?(?:(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\\bvon der\\b|\\bvan der\\b|\\bvan den\\b|du|\\bde la\\b|al|bin|ibn|el)?\\s?-?[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?)*\\b",
                "DLP_Strategy": "Detects full patient names using regex-based entity recognition."
            },
            "Patient_Number": {
                "Field_Identifier": [
                    "\\b(?:n[úu]mero|N\\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\\s?(?:de)?\\s?atendimento\\

## 2.2. Medical Prescription

### 2.2.1. Build pattern

In [110]:
# Accounted for the different ways in which medical prescription can be written (drugs, farmacology, administration etc)

Field_Identifier = r"\b(?:Prescriç[ãa]o|Receita M[ée]dica|Medicação|Drogas|Farmacologia|Medicamento|Uso Cont[íi]nuo|Administra[çc][ãa]o)\b"

# Did a research on the most common medication used in ICU context.
Content_Identifier = (
    r"\b(?:"
    # Analgesics and Sedatives
    r"Paracetamol|Dipirona|Ibuprofeno|Cetoprofeno|Tramadol|"
    r"Morfina|Fentanil|Sufentanil|Meperidina|Clonidina|"
    r"Midazolam|Lorazepam|Diazepam|Propofol|Ketamina|Dexmedetomidina|"

    # Vasopressors and Inotropes
    r"Dobutamina|Dopamina|Adrenalina|Noradrenalina|Vasopressina|"
    r"Milrinona|Fenilefrina|"

    # Antibiotics and antifungics
    r"Vancomicina|Meropenem|Imipenem|Piperacilina|Tazobactam|"
    r"Cefepima|Ceftriaxona|Ceftazidima|Levofloxacino|Ciprofloxacino|"
    r"Amicacina|Gentamicina|Linezolida|Fluconazol|Anfotericina|"
#
    # Cardiovascular Medications
    r"Amiodarona|Labetalol|Metoprolol|Esmolol|Nicardipina|"
    r"Hidralazina|Nitroprussiato|Diltiazem|"

    # Antipsychotics e Neuromuscular
    r"Haloperidol|Quetiapina|Olanzapina|Risperidona|"
    r"Pancurônio|Rocurônio|Cisatracúrio|Atracúrio|"

    # Other common medication
    r"Atorvastatina|Omeprazol|Pantoprazol|Ranitidina|"
    r"Heparina|Enoxaparina"
    r")\b"
)




### 2.2.2. Positive Stress-Test (PST)

In [111]:

pst_cases = [
    "Paciente em uso de Morfina para controle da dor.",
    "Prescrição de Paracetamol a cada 6 horas.",
    "Fentanil foi administrado em dose baixa.",
    "Dipirona e Ibuprofeno prescritos para febre.",
    "Paciente recebeu Midazolam para sedação profunda.",
    "Noradrenalina iniciada para suporte hemodinâmico.",
    "Antibiótico Vancomicina foi incluído na prescrição.",
    "Anfotericina indicada para tratar infecção fúngica.",
    "Hidralazina utilizada para controle da hipertensão.",
    "Dopamina administrada em infusão contínua.",
    "Quetiapina prescrita para agitação psicomotora.",
    "Rocurônio usado para indução anestésica.",
    "Paciente está tomando Omeprazol para proteção gástrica.",
    "Heparina foi administrada para evitar trombose.",
    "Metoprolol utilizado no controle da pressão arterial.",
    "Paciente recebe Enoxaparina como profilaxia antitrombótica.",
    "Labetalol foi prescrito para tratamento de crise hipertensiva.",
    "Midazolam foi administrado em protocolo de sedação contínua.",
    "Paciente apresentou boa resposta ao tratamento com Atorvastatina.",
    "Cefepima foi escolhida para tratamento de pneumonia grave."
]

# Apply regex
pst_results = ["✅ Matched" if re.search(Content_Identifier, case, re.IGNORECASE) else "❌ Missed" for case in pst_cases]

# Display results
df_pst_results = pd.DataFrame({"PST Test Case": pst_cases, "Detection Result": pst_results})
df_pst_results


,PST Test Case,Detection Result
0,Paciente em uso de Morfina para controle da dor.,✅ Matched
1,Prescrição de Paracetamol a cada 6 horas.,✅ Matched
2,Fentanil foi administrado em dose baixa.,✅ Matched
3,Dipirona e Ibuprofeno prescritos para febre.,✅ Matched
4,Paciente recebeu Midazolam para sedação profunda.,✅ Matched
5,Noradrenalina iniciada para suporte hemodinâmico.,✅ Matched
6,Antibiótico Vancomicina foi incluído na prescr...,✅ Matched
7,Anfotericina indicada para tratar infecção fún...,✅ Matched
8,Hidralazina utilizada para controle da hiperte...,✅ Matched
9,Dopamina administrada em infusão contínua.,✅ Matched


### 2.2.3. Negative Stress-Test (NST)

In [112]:
nst_cases = [
    "O hospital recebeu uma nova remessa de seringas.",
    "Equipe de enfermagem realizou treinamento em cuidados intensivos.",
    "Lista de equipamentos disponíveis no setor de emergência.",
    "Reunião com fornecedores para aquisição de novos insumos.",
    "Procedimentos administrativos para solicitação de medicamentos.",
    "Paciente não apresentou necessidade de medicação adicional.",
    "A farmácia hospitalar organiza os estoques semanalmente.",
    "Discussão sobre melhores práticas em prescrição médica.",
    "Professor de farmacologia apresenta nova pesquisa.",
    "Relatório sobre uso racional de medicamentos na UTI.",
    "Insumos médicos estão disponíveis para enfermagem.",
    "Política de segurança hospitalar reforçada.",
    "Gestão hospitalar planeja novas estratégias para atendimento.",
    "Pesquisa avalia eficácia de tratamento não farmacológico.",
    "Artigo publicado sobre tendências em terapias médicas.",
    "Especialistas discutem avanços na área de anestesiologia.",
    "A farmácia clínica é essencial para otimizar prescrições.",
    "Engenharia hospitalar realiza manutenção em ventiladores.",
    "Comissão de controle de infecção analisa novos protocolos.",
    "Hospital inaugura novo laboratório de análises clínicas."
]

# Apply regex
nst_results = ["✅ Matched" if re.search(Content_Identifier, case, re.IGNORECASE) else "❌ Correctly Ignored" for case in nst_cases]

# Display results
df_nst_results = pd.DataFrame({"NST Test Case": nst_cases, "Detection Result": nst_results})
df_nst_results.style

,NST Test Case,Detection Result
0,O hospital recebeu uma nova remessa de seringas.,❌ Correctly Ignored
1,Equipe de enfermagem realizou treinamento em cuidados intensivos.,❌ Correctly Ignored
2,Lista de equipamentos disponíveis no setor de emergência.,❌ Correctly Ignored
3,Reunião com fornecedores para aquisição de novos insumos.,❌ Correctly Ignored
4,Procedimentos administrativos para solicitação de medicamentos.,❌ Correctly Ignored
5,Paciente não apresentou necessidade de medicação adicional.,❌ Correctly Ignored
6,A farmácia hospitalar organiza os estoques semanalmente.,❌ Correctly Ignored
7,Discussão sobre melhores práticas em prescrição médica.,❌ Correctly Ignored
8,Professor de farmacologia apresenta nova pesquisa.,❌ Correctly Ignored
9,Relatório sobre uso racional de medicamentos na UTI.,❌ Correctly Ignored


### 2.2.4. Moriarty Stress-Test (MST)

In [113]:


# Moriarty Stress-Test (MST) Cases for Prescription Drugs
mst_cases_prescription = [
    # **1️⃣ Spacing Manipulation**
    "P a r a c e t a m o l 500mg",
    "A d r e n a l i n a - 1mg/ml",

    # **2️⃣ Special Character Obfuscation**
    "D*o*p*a*m*i*n*a",  # Inserted asterisks
    "M3r0p3n3m",  # Leetspeak attempt

    # **3️⃣ Encoding Tricks**
    "P%61r%61c%65t%61m%6f%6c",
    "Fluconazol".encode('utf-16', 'surrogatepass').decode('utf-16'),

    # **4️⃣ Homoglyph Substitutions**
    "Pαrαcetamol",  # Greek Alpha instead of 'A'
    "Enoҳaparina",  # Cyrillic 'ҳ' instead of 'x'

    # **5️⃣ Multi-line Injection**
    "Paciente foi prescrito:\nIbuprofeno\n500mg",

    # **6️⃣ Contextual Evasion**
    "Paciente recebeu um analgésico para dor leve.",
    "Este estudo analisa os efeitos do Midazolam em ratos.",

    # **7️⃣ Drug Combinations (Complex Cases)**
    "Foi administrado Dopamina junto com Adrenalina 2mg.",
    "O tratamento incluiu Ceftriaxona e um antifúngico não especificado.",
]

# Apply MST regex testing
mst_results_prescription = [
    "✅ Matched" if re.search(Content_Identifier, case, re.IGNORECASE) else "❌ Evaded"
    for case in mst_cases_prescription
]

# Create MST results DataFrame
df_mst_results_prescription = pd.DataFrame({"MST Test Case": mst_cases_prescription, "Detection Result": mst_results_prescription})
df_mst_results_prescription

,MST Test Case,Detection Result
0,P a r a c e t a m o l 500mg,❌ Evaded
1,A d r e n a l i n a - 1mg/ml,❌ Evaded
2,D*o*p*a*m*i*n*a,❌ Evaded
3,M3r0p3n3m,❌ Evaded
4,P%61r%61c%65t%61m%6f%6c,❌ Evaded
5,Fluconazol,✅ Matched
6,Pαrαcetamol,❌ Evaded
7,Enoҳaparina,❌ Evaded
8,Paciente foi prescrito:\nIbuprofeno\n500mg,✅ Matched
9,Paciente recebeu um analgésico para dor leve.,❌ Evaded


### 2.2.5. Time Stress-Test (TST)

#### **TST Goals**  
- **Monitor drug evolution** – Ensure newly introduced medications are captured while outdated drugs are deprecated.  
- **Detect shifts in prescribing trends** – Track how ICU medication protocols change over time.  
- **Ensure compliance with regulatory updates** – Align detection with evolving controlled substance lists (e.g., ANVISA, FDA, EMA).  

#### **TST Implementation**  
 **Periodic Drug List Updates** → Compare regex matches against the latest formularies, clinical guidelines, and ICU prescribing patterns.  
 **Historical Frequency Tracking** → Analyze term frequency in medical records to identify trends in commonly prescribed ICU drugs.  
 **Classification Stability** → Assign an adaptability index to each identifier, setting **high-risk medications** for frequent review cycles.  



### 2.2.6. Add Identifier to Dict

In [114]:
# Initialize ICU Dictionary if not already present
icu_data_dict_granular = icu_data_dict_granular if 'icu_data_dict_granular' in locals() else {}

# Define categories and their respective identifiers
icu_categories = [
    ("Prescriptions", "Medical_Prescription", Field_Identifier, Content_Identifier, 
     "Detects prescribed medications, controlled substances, and dosage information.")
]

# Add each field identifier, content regex, and DLP strategy to the dictionary
for category, field, field_identifier, content_identifier, dlp_strategy in icu_categories:
    icu_data_dict_granular.setdefault(category, {}).setdefault("Fields", {})[field] = {
        "Field_Identifier": field_identifier,
        "Content_Identifier": content_identifier,
        "DLP_Strategy": dlp_strategy
    }

# Print the structured ICU dictionary
import json
print(json.dumps(icu_data_dict_granular, indent=4, ensure_ascii=False))


{
    "Patient_Identifiers": {
        "Fields": {
            "Complete_Name": {
                "Field_Identifier": [
                    "\\b(?:[Nn]ome)\\b",
                    "[nN]ome\\s?[cC]ompleto",
                    "\\b[Nn]ome\\s?(?:d[oa])?\\s?[Pp]aciente\\b",
                    "[Pp]aciente"
                ],
                "Content_Identifier": "\\b[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?-?(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\\bvon der\\b|\\bvan der\\b|\\bvan den\\b|du|\\bde la\\b|al|bin|ibn|el)?\\s?[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?-?(?:(?:da|de|do|dos|das|di|del|della|degli|dei|von|van|\\bvon der\\b|\\bvan der\\b|\\bvan den\\b|du|\\bde la\\b|al|bin|ibn|el)?\\s?-?[A-ZÀ-Ÿ][a-zà-ÿ]+\\s?)*\\b",
                "DLP_Strategy": "Detects full patient names using regex-based entity recognition."
            },
            "Patient_Number": {
                "Field_Identifier": [
                    "\\b(?:n[úu]mero|N\\.?|#|Nº|I[Dd]|C[óo]digo|identifica(?:ção|cao))\\s?(?:de)?\\s?atendimento\\

# 3. Export Dict

In [115]:
import json

# Define the file path
json_file_path = "icu_data_dict_granular.json"

# Export the dictionary to a JSON file
with open(json_file_path, "w", encoding="utf-8") as f:
    json.dump(icu_data_dict_granular, f, indent=4, ensure_ascii=False)

print(f"Dictionary successfully saved as {json_file_path}")


Dictionary successfully saved as icu_data_dict_granular.json
